In [5]:
import dropbox
import urllib.request, json 
from collections import defaultdict
import datetime as DT
import ast
from __future__ import print_function
from __future__ import division
import collections
import nltk
import numpy as np
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

# Helper libraries
from w266_common import utils, vocabulary

    

In [ ]:
##Download the pre-processed tweets from dropbox

access_token = 'p3Ze8FpdRTAAAAAAAAAADb9hCQ8aXXYU3A3gGM1HEXyeMrs8KI2SrA71KDmmCXte'
dbx = dropbox.Dropbox(access_token)
metadata, res = dbx.files_download(path="/Inputdata2.txt")


access_token2 = 'm_fKi8mWZ0AAAAAAAAAADlfzTm37f1y45k92Xpiw1B6mjT3dQqzNvKgpIUSR17uq'
dbx2 = dropbox.Dropbox(access_token2)

#Write the combined file into the drive
with open("InputX.txt", "wb") as f:
    metadata, res = dbx.files_download(path="/Inputdata2.txt")
    metadata2, res2 = dbx2.files_download(path="/Inputdata.txt")
    f.write(res.content)
    f.write(res2.content)

In [6]:
##Translate date of the tweet to quarter reference
def Quarterlookup(date):
    if date>=DT.datetime(2018,10,1):
        return '4Q18'
    elif date>=DT.datetime(2018,7,1):
        return '3Q18'
    elif date>=DT.datetime(2018,4,1):
        return '2Q18'
    elif date>=DT.datetime(2018,1,1):
        return '1Q18'
    elif date>=DT.datetime(2017,10,1):
        return '4Q17'
    elif date>=DT.datetime(2017,7,1):
        return '3Q17'
    elif date>=DT.datetime(2017,4,1):
        return '2Q17'
    elif date>=DT.datetime(2017,1,1):
        return '1Q17'
    elif date>=DT.datetime(2016,10,1):
        return '4Q16'
    elif date>=DT.datetime(2016,7,1):
        return '3Q16'
    elif date>=DT.datetime(2016,4,1):
        return '2Q16'
    elif date>=DT.datetime(2016,1,1):
        return '1Q16'
    elif date>=DT.datetime(2015,10,1):
        return '4Q15'
    elif date>=DT.datetime(2015,7,1):
        return '3Q15'
    elif date>=DT.datetime(2015,4,1):
        return '2Q15'
    else:
        return '1Q15'

In [ ]:
##Reading the Input X file and process it
d=defaultdict(list)
x_tokens=[]
with open('InputX.txt') as f:
    for line in f:
        count=0
        temp =-1
        k,v=line.split(':',1)
        while True:
            
            oldtemp=temp
            temp=v.find(']]',temp+1)
            if count==0:
                d[k].append(v[oldtemp+2:temp+2]) 
            else:
                d[k].append(v[oldtemp+4:temp+2])
                x_tokens.append(tokenizer.tokenize(v[oldtemp+4:temp+2]))
             
            count+=1
            if temp == -1: break
         
    

In [ ]:
allword=set()
for i in x_tokens:
    for j in i:
        allword.add(j)
allword=sorted(allword)
vocab = vocabulary.Vocabulary(allword, size=None)  # size=None means unlimited
print("Vocabulary size: {:,}".format(vocab.size))


In [ ]:
##Converting the string of data into dictionary
InputX=defaultdict(dict)
InputN=defaultdict(dict)

for key,value in d.items():
    Inputx=defaultdict(list)
    Inputn=defaultdict(list)
    for value2 in value:
        
        temp=value2.find('), ')
        date=value2[14:temp+3]
        try:
            year=int(date[1:5])
            month=int(date[date.find(',')+2:date.find(',',date.find(',')+1)])
            day=int(date[date.find(')')-2:date.find(')')])
            Inputx[Quarterlookup(DT.datetime(year,month,day))].append(vocab.words_to_ids(ast.literal_eval(value2[temp+3:-1])))
            Inputn[Quarterlookup(DT.datetime(year,month,day))].append(len(ast.literal_eval(value2[temp+3:-1])))
        except:
            pass
    InputX[key]=Inputx
    InputN[key]=Inputn

In [11]:
##List of stockname and relevant stock code
l = [("CVSHealth", "CVS"),
("RiteAid", "RAD"),
("Conn's", "CONN"),
("Casey's", "CASY"),
("Kroger", "KR"),
("VillageSuperMarket", "VLGEA"),
("WeisMarkets", "WMK"),
("HavertyFurniture", "HVT"),
("Kirkland's", "KIRK"),
("Pier1Imports","PIR"),
("BuildersFirstSource","BLDR"),
("HuttigBuildingProducts", "HBP"),
("LumberLiquidators", "LL"),
("TransWorldEntertainment", "TWMC"),
("MarineMax", "HZO"),
("Medifast", "MED"),
("SallyBeauty", "SBH"),
("Sotheby's", "BID"),
("StarGasPartners", "SGU"),
("SuburbanPropanePartners", "SPH"),
("TitanMachinery", "TITN"),
("TractorSupply","TSCO"),
("TravelCentersofAmerica", "TA"),
("Big5SportingGoods", "BGFV"),
("Dick'sSportingGoods", "DKS"),
("HibbettSports", "HIBB"),
("Build-A-BearWorkshop", "BBW")]
     
all_stock = {}
[all_stock.update({k:v}) for k,v in l]
print(all_stock)

{'CVSHealth': 'CVS', 'RiteAid': 'RAD', "Conn's": 'CONN', "Casey's": 'CASY', 'Kroger': 'KR', 'SUPERVALU': 'SVU', 'VillageSuperMarket': 'VLGEA', 'WeisMarkets': 'WMK', 'HavertyFurniture': 'HVT', "Kirkland's": 'KIRK', 'Pier1Imports': 'PIR', 'BuildersFirstSource': 'BLDR', 'HuttigBuildingProducts': 'HBP', 'LumberLiquidators': 'LL', 'TransWorldEntertainment': 'TWMC', 'MarineMax': 'HZO', 'Medifast': 'MED', 'SallyBeauty': 'SBH', "Sotheby's": 'BID', 'StarGasPartners': 'SGU', 'SuburbanPropanePartners': 'SPH', 'TitanMachinery': 'TITN', 'TractorSupply': 'TSCO', 'TravelCentersofAmerica': 'TA', 'Big5SportingGoods': 'BGFV', "Dick'sSportingGoods": 'DKS', 'HibbettSports': 'HIBB', 'Build-A-BearWorkshop': 'BBW'}


In [18]:
##Read the quarterly return detail from edgaronline and convert into library
top="http://datafied.api.edgar-online.com/v2/corefinancials/qtr?primarysymbols="
bottom="&appkey=asc97xrhkyu4959aptu76zxj"
Outputy =defaultdict(dict)
for stockname,stock in all_stock.items():
    check=top+stock+bottom
    with urllib.request.urlopen(check) as url:
        data = json.loads(url.read().decode())
    count1=0
    count2=0
    end_date=[]
    revenue_change=[]
    last_rev=0

    for key, value in data.items():
        
        for key2, value2 in value.items():          
        
            if count1!=0:  
                for i in reversed(value2):
                  
                    for j in i["values"]:
                     
                        if count2!=0:   
                            if j["field"] =="periodenddate":
                                end_date.append(Quarterlookup(DT.datetime.strptime(j['value'], '%m/%d/%Y')))
                       
                            ##Convert revenue into the y_label of 0 for negative growth and 1 for positive growth
                            if j["field"] =="totalrevenue":
                                revenue_cal =round(float(j['value'])/float(last_rev) - 1,3)
                                if revenue_cal>0:
                                    revenue_change.append(1)
                                else:
                                    revenue_change.append(0)
                        else:
                            
                            if j["field"] =="totalrevenue":      
                                last_rev=j['value']
                              
                    count2+=1
            count1+=1
            
    ##Output will be a dictionary with stockname
    Outputy[stockname]["period"]=end_date
    Outputy[stockname]["revenuechange"]=revenue_change
        

In [65]:
with open('Outputy.txt', 'w') as f:
    print(Outputy, file=f)

In [63]:
InputXTrain=defaultdict(dict)
InputXTest=defaultdict(dict)
InputNTrain=defaultdict(dict)
InputNTest=defaultdict(dict)
OutputyTrain=defaultdict(dict)
OutputyTest=defaultdict(dict)

stock =['CVSHealth', 'RiteAid', "Conn's", "Casey's", 'Kroger', 'VillageSuperMarket', 'WeisMarkets', 'HavertyFurniture', "Kirkland's", 'Pier1Imports', 'BuildersFirstSource', 'HuttigBuildingProducts', 'LumberLiquidators', 'TransWorldEntertainment', 'MarineMax', 'Medifast', 'SallyBeauty',"Sotheby's", 'StarGasPartners', 'SuburbanPropanePartners', 'TitanMachinery', 'TractorSupply', 'TravelCentersofAmerica', 'Big5SportingGoods', "Dick'sSportingGoods", 'HibbettSports', 'Build-A-BearWorkshop']
quarter=['1Q15','2Q15','3Q15','4Q15','1Q16','2Q16','3Q16','4Q16','1Q17','2Q17','3Q17','4Q17','1Q18','2Q18','3Q18']

for i in stock:
    for j in range(len(quarter)):
        try:
            if quarter[j]=='3Q18':
                InputXTest[i][quarter[j]]= InputX [i][quarter[j]]
                InputNTest[i][quarter[j]]= InputN [i][quarter[j]]
                OutputyTest[i][quarter[j]] =Outputy [i]['revenuechange'][j]
            else:
                InputXTrain [i][quarter[j]]= InputX [i][quarter[j]]
                InputNTrain [i][quarter[j]]= InputN [i][quarter[j]]
                OutputyTrain [i][quarter[j]] =Outputy [i]['revenuechange'][j]
        except:
            pass

CVSHealth
{'1Q15': 0, '2Q15': 1, '3Q15': 1, '4Q15': 1, '1Q16': 1, '2Q16': 1, '3Q16': 1, '4Q16': 1, '1Q17': 1, '2Q17': 1, '3Q17': 1, '4Q17': 1, '1Q18': 1, '2Q18': 1}
RiteAid
{'1Q15': 1, '2Q15': 0, '3Q15': 1, '4Q15': 1, '1Q16': 0, '2Q16': 1, '3Q16': 1, '4Q16': 0, '1Q17': 0, '2Q17': 0, '3Q17': 0, '4Q17': 0, '1Q18': 0, '2Q18': 0}
Conn's
{'1Q15': 1, '2Q15': 0, '3Q15': 1, '4Q15': 1, '1Q16': 1, '2Q16': 1, '3Q16': 1, '4Q16': 1, '1Q17': 1, '2Q17': 0, '3Q17': 0, '4Q17': 1, '1Q18': 1, '2Q18': 0}
Casey's
{'1Q15': 0, '2Q15': 0, '3Q15': 0, '4Q15': 0, '1Q16': 0, '2Q16': 0, '3Q16': 0, '4Q16': 0, '1Q17': 0, '2Q17': 0, '3Q17': 0, '4Q17': 1, '1Q18': 0, '2Q18': 0}
Kroger
{'1Q15': 1, '2Q15': 1, '3Q15': 1, '4Q15': 1, '1Q16': 1, '2Q16': 1, '3Q16': 1, '4Q16': 1, '1Q17': 1, '2Q17': 1, '3Q17': 1, '4Q17': 1, '1Q18': 1, '2Q18': 1}
WeisMarkets
{'1Q15': 0, '2Q15': 1, '3Q15': 0, '4Q15': 1, '1Q16': 1, '2Q16': 1, '3Q16': 1, '4Q16': 1, '1Q17': 1, '2Q17': 1, '3Q17': 1, '4Q17': 1, '1Q18': 1, '2Q18': 1}
HavertyFurniture
{